In [ ]:
using ModelingToolkit, MethodOfLines, OrdinaryDiffEq, DomainSets, Plots

### To use `MethodOfLines.jl`, we first need to symbolically define the system as a `PDESystem`
---

In [ ]:
# add independent variables
@parameters x y t 

# now add dependent variables. 
# the ".." indicates it's a function of all independent variables 
@variables u(..) v(..)

# Now add the discretization matrices you need
Dt = Differential(t)
Dx = Differential(x)
Dy = Differential(y)
Dxx = Differential(x)^2
Dyy = Differential(y)^2
∇²(u) = Dxx(u) + Dyy(u)

# add a forcing function "f"
brusselator_f(x, y, t) = (((x-0.3)^2 + (y-0.6)^2) <= 0.1^2) * (t >= 1.1) * 5.

# define the domain bounds
# in this case, (x, y) ∈ [0,1] 
x_min = y_min = t_min = 0.0
x_max = y_max = 1.0

# and t ∈ [0, 11.5]
t_max = 11.5

# parameter value
α = 10.

# initial conditions
u0(x,y,t) = 22(y*(1-y))^(3/2)
v0(x,y,t) = 27(x*(1-x))^(3/2)

# define the equations in a vector like this
eq = [Dt(u(x,y,t)) ~ 1. + v(x,y,t)*u(x,y,t)^2 - 4.4*u(x,y,t) + α*∇²(u(x,y,t)) + brusselator_f(x, y, t),
       Dt(v(x,y,t)) ~ 3.4*u(x,y,t) - v(x,y,t)*u(x,y,t)^2 + α*∇²(v(x,y,t))]

# now throw the domain parameters in a list like this (1 per dimension)
domains = [x ∈ Interval(x_min, x_max),
              y ∈ Interval(y_min, y_max),
              t ∈ Interval(t_min, t_max)]

# add periodic BCs for each dependent variable
bcs = [u(x,y,0) ~ u0(x,y,0), # for u at t=0, use its initial condition
       u(0,y,t) ~ u(1,y,t), # u at x=0 is the same as u at x = 1 (end of x-domain)
       u(x,0,t) ~ u(x,1,t), # u at y=0 is the same as u at x = 1 (end of y-domain)

       v(x,y,0) ~ v0(x,y,0), # for u at t=0, use its initial condition
       v(0,y,t) ~ v(1,y,t), # u at x=0 is the same as u at x = 1 (end of x-domain)
       v(x,0,t) ~ v(x,1,t)] # u at y=0 is the same as u at x = 1 (end of y-domain)

# combine everything into 1 PDESystem object
@named pdesys = PDESystem(eq,bcs,domains,[x,y,t],[u(x,y,t),v(x,y,t)]);

### Now, we define the discretization
---
1. If you want to leave t undiscretized, just supply t as an argument.
2. If you want to discretize in t, just remove the argument `t` and supply `t => dt` in the dxs.

In [ ]:
Nx = 32
Ny = 32

# Now create a second-order accurate finite difference method
order = 2 # This may be increased to improve accuracy of some schemes

# Integers for x and y are interpreted as number of points. Use a Float to directly specify stepsizes dx and dy.
discretization = MOLFiniteDifference([x => Nx, y => Ny], t, approx_order=order);

### Next, we discretize the system, converting the PDESystem in to an ODEProblem
---

In [ ]:
# Convert the PDE problem into an ODE problem
println("Discretization:")
@time prob = discretize(pdesys,discretization)

In [ ]:
println("Solve:")
@time sol = solve(prob, TRBDF2(), saveat=0.1) # solver for large stiff systems

In [ ]:
discrete_x = sol[x]
discrete_y = sol[y]
discrete_t = sol[t]

solu = sol[u(x, y, t)]
solv = sol[v(x, y, t)]

In [ ]:
anim = @animate for k in 1:length(discrete_t)
    heatmap(solu[2:end, 2:end, k], title="$(discrete_t[k])") # 2:end since end = 1, periodic condition
end
gif(anim, "Brusselator2Dsol_u.gif", fps = 8)

In [ ]:
anim = @animate for k in 1:length(discrete_t)
    heatmap(solv[2:end, 2:end, k], title="$(discrete_t[k])")
end
gif(anim, "Brusselator2Dsol_v.gif", fps = 8)